In [1]:
# -*- coding: utf-8 -*-
import os
import re
import csv
import sys
import glob
import errno
import random
import sqlite3
import subprocess
import pandas as pd
import pymysql.cursors
import gensim, logging
from __future__ import division
from gensim.models import FastText
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from abbreviations import get_abbreviations

In [2]:
# Connect to db
def connect_to_db():
    database = "db/sentences.db"
    conn = create_connection(database)
    return conn

In [3]:
# Connect to DB
def create_connection(db_file):
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
 
    return None

In [4]:
#cleanup tokens
def cleanup_token(tkn):
    new_tkn = tkn.strip().strip(',').strip('.')
    if len(new_tkn)>=2 and new_tkn[0] == '(' and new_tkn[len(new_tkn)-1]==')':
        new_tkn=new_tkn.rstrip(')').lstrip('(')
    return new_tkn

In [5]:
def get_sentences_from_db(conn):
    sentences = []
    cur = conn.cursor()
    cur.execute("select sentence, haspolys, id, docid, processed_sentence from sentences")
    rows = cur.fetchall()
    for row in rows:
        # Use regex to preserve spaces between parentheses
        #sentences.append(row[0].rstrip(".").split())
        tokens = re.split(r"\s+(?=[^()]*(?:\(|$))", row[0].rstrip("."))
        new_tokens = list(map(lambda token:cleanup_token(token),tokens))
        sentences.append(new_tokens)
    return sentences

In [6]:
def get_sentences_for_dois(doi,conn):
    sentences = []
    cur = conn.cursor()
    query = "select sentence from sentences where docid='%s'" % (doi)
    cur.execute(query)
    rows = cur.fetchall()
    for row in rows:
        # Use regex to preserve spaces between parentheses
        #sentences.append(row[0].rstrip(".").split())
        #tokens = re.split(r"\s+(?=[^()]*(?:\(|$))", row[0].rstrip("."))
        #new_tokens = list(map(lambda token:cleanup_token(token),tokens))
        #sentences.append(new_tokens)
        sentences.append(row[0].rstrip("."))

    return sentences

In [7]:
def generate_gensim_model(modeltype, sentences):
    if modeltype == 'skipgram': 
        model = gensim.models.Word2Vec(sentences, min_count=1, sg=1, sorted_vocab=1)
        modelname = "models/gensim_skipgram.bin"
    elif modeltype == 'cbow':
        model = gensim.models.Word2Vec(sentences, min_count=1, sorted_vocab=1)
        modelname = "models/gensim_cbow.bin"
    else:
        print('Unknown model type: %s' % modeltype)
        return(None)       
    model.save(modelname)
    return model

In [8]:
# Load gensim model
def load_gensim_model(modeltype):
    if modeltype == 'cbow':
        model_file = './models/gensim_cbow.bin'
        if os.path.exists(model_file):
            model = gensim.models.Word2Vec.load(model_file)
        else:
            print('Model file does not exist: %s' % model_file)
    elif modeltype == 'skipgram':
        model_file = './models/gensim_skipgram.bin'
        if os.path.exists(model_file):
            model = gensim.models.Word2Vec.load(model_file)
        else:
            print('Model file does not exist: %s' % model_file)
    else:
        print('Unknown model type: %s' % modeltype)
        return(None)
    return model

In [9]:
regenerate_gensim_model_flag = False
model_type = 'cbow'

In [10]:
# Generate or load gensim model. 
if regenerate_gensim_model_flag:
    # Connect to db
    connection = connect_to_db()
    # Get sentences from db
    sentences = get_sentences_from_db(connection)
    print len(sentences)
    model = generate_gensim_model(model_type, sentences)
else:
    model = load_gensim_model(model_type)
    
labels = list(model.wv.vocab) 


In [11]:
def get_polymers(doi_input_file, entire_document):
#     # Most frequent polymer is polystyrene, we compare words vectors in sentences to 3 forms of this polymer name
#     frequent_polymers = ["polystyrene","poly(styrene)","PS"]
#     print('Using these frequent polymers: %s' % str(frequent_polymers))
#     #frequent_polymers = ["polystyrene","PS","poly(methyl methacrylate)"]
    
    # Open ground truth document
    #f = open('evaluation/classifier_dois.txt','rb')
    f = open(doi_input_file,'rb')
    lines = f.readlines()
    print('Processing %d documents as listed in %s' % (len(lines), doi_input_file))
    sentences2 = []
    connection = connect_to_db()
    
    # Process line in prediction file
    for line in lines:

        # Read the doi
        doi = line.split("\t")[0].strip("\n")
         
        if entire_document == 1:
            sentences = get_sentences_for_dois(doi,connection)
            
        for sentence in sentences: 
            sentences2.append(sentence)

    return sentences2

In [12]:
#get sentences from documents
sentences_list = get_polymers('data/classifier_dois.txt', 1)

Processing 50 documents as listed in data/classifier_dois.txt


In [13]:
#get dictionary of abbreviations:polymers
abbrs = get_abbreviations(sentences_list)

In [14]:
#TR is total polymers that are in both vocabulary and longnames
TP = 0
#S is if acronym is found in most similar words
S = 0


#determine how often acronym is found as a most similar word
for key in abbrs.keys(): 
    longname = abbrs[key]
    if longname in labels: 
        TP = TP + 1
        most_similar_results = model.wv.most_similar(positive=longname, topn = 10)        
        for item in most_similar_results: 
            if key in item: 
                S = S + 1                
P = S/TP


print(TP)
print(S)
print(P)

/anaconda2/lib/python2.7/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


129
20
0.15503875969
